In [113]:
from __future__ import division

In [121]:
import pandas as pd
import random
import csv

In [39]:
child_wishlist_cols = ["child"] + ["gift_" + str(i) for i in range(0, 100)]
child_wishlist = pd.read_csv("./data/child_wishlist_v2.csv", names=child_wishlist_cols)

In [40]:
gift_goodkids_cols = ["gift"] + ["child_" + str(i) for i in range(0, 1000)]
gift_goodkids = pd.read_csv("./data/gift_goodkids_v2.csv", names=gift_goodkids_cols)

In [41]:
child_prefs = {}
for row_iter, row in child_wishlist.iterrows():
    child_prefs[row.child] = row.iloc[1:].tolist()

gift_prefs = {}
gift_inventory = {}
for row_iter, row in gift_goodkids.iterrows():
    gift_prefs[row.gift] = row.iloc[1:].tolist()
    gift_inventory[row.gift] = 1000

In [96]:
siblings_dict = {}

for i in range(0, 5000, 3):
    siblings_dict[i] = [i+1, i+2]
    siblings_dict[i+1] = [i, i+2]    
    siblings_dict[i+2] = [i, i+1]
    
for i in range(5001, 45000, 2):
    siblings_dict[i] = [i+1]
    siblings_dict[i+1] = [i]    

for i in range(45001, len(child_prefs)):
    siblings_dict[i] = []

In [97]:
max_child_score = 200
max_santa_score = 2000

num_children = len(child_prefs)
num_gifts = len(gift_prefs)

In [98]:
def get_child_happiness_score(child_id, gift_id):
    
    prefs = child_prefs[child_id]
    
    if gift_id not in prefs:
        return -1
    else:
        return 2*(len(prefs) - prefs.index(gift_id))

In [99]:
def get_santa_happiness_score(child_id, gift_id):
    
    prefs = gift_prefs[gift_id]
    
    if child_id not in prefs:
        return -1
    else:
        return 2*(len(prefs) - prefs.index(child_id))

In [100]:
def get_normalized_happiness(child_gift_dict):

    normalized_ch = 0
    normalized_sh = 0
    
    for child_id in child_gift_dict:

        child_happiness_score = get_child_happiness_score(child_id, child_gift_dict[child_id])
        normalized_ch += child_happiness_score/max_child_score

        santa_happiness_score = get_santa_happiness_score(child_id, child_gift_dict[child_id])    
        normalized_sh += santa_happiness_score/max_santa_score
    
    return normalized_ch, normalized_sh

In [101]:
def get_score(anch, ansh):
    
    return pow(anch, 3) + pow(ansh, 3)

In [156]:
gift_inventory = {}
for row_iter, row in gift_goodkids.iterrows():
    gift_inventory[row.gift] = 1000

In [ ]:
total_ch = 0
total_sh = 0

children_processed = []

child_gift_res = {}

for child_id in child_prefs:
# for child_id in range(854629, len(child_prefs)):

    if child_id in children_processed:
        continue

    siblings = siblings_dict[child_id]

    children_family = [child_id] + siblings
    num_children_family = len(children_family)

    prefs = []
    for child_family in children_family:
        prefs += child_prefs[child_family]

    max_score = -32768
    max_ch = -32768
    max_sh = -32768
    
    for pref in prefs:

        if gift_inventory[pref] < num_children_family:
            continue

        child_gift_dict = {child_family:pref for child_family in children_family}

        normalized_ch, normalized_sh = get_normalized_happiness(child_gift_dict)
        
        local_anch = (total_ch + normalized_ch)/(len(children_processed) + num_children_family)
        local_ansh = (total_sh + normalized_sh)/(len(children_processed) + num_children_family)
        local_score = get_score(local_anch, local_ansh)
        
        if local_score > max_score:
            max_score = local_score
            max_ch = total_ch + normalized_ch
            max_sh = total_sh + normalized_sh
            final_pref = pref
    
    if max_score == -32768: #the child's preferred gifts' inventory is finished
        
        min_child_index = 32768

        for g in gift_prefs:
            
            if gift_inventory[g] > 0 and child_id in gift_prefs[g]:
                child_index = gift_prefs[g].index(child_id)
            
                if child_index < min_child_index:
                    
                    min_child_index = child_index
                    final_pref = g
        
        if min_child_index == 32768:
            #just selecting a random gift whose inventory is greater than 0.
            #The child's preferred gifts are maxed out and no gift that is available has the child in its list.
            final_pref = random.choice([key for (key, val) in gift_inventory.items() if val > 0])            
            
        child_gift_dict = {child_family:final_pref for child_family in children_family}

        normalized_ch, normalized_sh = get_normalized_happiness(child_gift_dict)
        
        local_anch = (total_ch + normalized_ch)/(len(children_processed) + num_children_family)
        local_ansh = (total_sh + normalized_sh)/(len(children_processed) + num_children_family)
        local_score = get_score(local_anch, local_ansh)

        max_score = local_score
        max_ch = total_ch + normalized_ch
        max_sh = total_sh + normalized_sh

    print child_id, final_pref, max_score
    total_ch = max_ch
    total_sh = max_sh
    gift_inventory[final_pref] -= num_children_family
    
    for child_family in children_family:
        child_gift_res[child_family] = final_pref
    
    children_processed += children_family

0 707 0.254037036912
3 360 0.219620036912
6 915 0.223894265479
9 748 0.286951843553
12 945 0.262964053505
15 672 0.238648476748
18 474 0.261267264817
21 542 0.262656333281
24 613 0.284601766679
27 396 0.279298765838
30 642 0.287892181721
33 917 0.2959260801
36 886 0.295101325034
39 965 0.304788865948
42 130 0.302862734009
45 618 0.300062138731
48 564 0.294469996008
51 445 0.286649831393
54 373 0.293846936273
57 213 0.28946065825
60 93 0.285529927285
63 255 0.283161962838
66 653 0.279707667939
69 46 0.274448995821
72 647 0.278189309827
75 223 0.282168025499
78 663 0.279583716096
81 334 0.27704636733
84 318 0.27484359233
87 603 0.275541434048
90 273 0.272314574065
93 744 0.267751514787
96 423 0.264759193992
99 780 0.264742657452
102 383 0.264138522758
105 446 0.264483368854
108 445 0.263474677545
111 988 0.263601851951
114 95 0.266632971759
117 805 0.267764496888
120 7 0.267474557769
123 234 0.265621358597
126 721 0.267852665514
129 149 0.267203109661
132 340 0.269215358433
135 829 0.268

In [136]:
with open("greedy_sub_1.csv", 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["ChildId", "GiftId"])
    writer.writeheader()

    for key, value in child_gift_res.items():
        writer.writerow({'ChildId': key, 'GiftId': value})